In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#For accessing files
f = open('accessfiles.py', 'wb')
f.write(streaming_body_1.read())

import accessfiles

In [1]:
!pip install pyspark==2.4.5


In [2]:
from zipfile import ZipFile
import time
import numpy as np
from operator import add

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

from accessfiles import load_files

load_files()

In [3]:

ratingsRDD=sc.textFile("ratings.dat")
movies=ratingsRDD.map(lambda line:int(line.split("::")[1]))
movies_pair=movies.map(lambda mv:(mv,1))
movies_count=movies_pair.reduceByKey(add)
movies_sorted=movies_count.sortBy(lambda x: x[1], False, 1)         
mv_top10List=movies_sorted.take(10)
mv_top10RDD=sc.parallelize(mv_top10List)

mv_names=sc.textFile("movies.dat").map(lambda line:(int(line.split("::")[0]),line.split("::")[1]))
print(mv_top10RDD.collect())

join_out=mv_names.join(mv_top10RDD)
top10_rdd = join_out.sortBy(lambda x: x[1][1],False).map(lambda x: (x[0],x[1][0],x[1][1])).repartition(1)
top10_df = top10_rdd.toDF().toPandas()
top10_df.columns=['Movie_ID','Name','Views']

[(2858, 3428), (260, 2991), (1196, 2990), (1210, 2883), (480, 2672), (2028, 2653), (589, 2649), (2571, 2590), (1270, 2583), (593, 2578)]


In [4]:
from accessfiles import create_download_link
create_download_link(top10_df, filename='Most_viewed_top10movies.csv')